In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/

/content/gdrive/MyDrive


In [3]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from  torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [4]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [6]:
def model(t_u, w, b):
  return w*t_u + b

def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

In [7]:
params = torch.tensor([1.0, 0.0], requires_grad = True)

In [21]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
  for epoch in range(1, n_epochs + 1):
    if params.grad is not None:
      params.grad.zero_()
    t_p = model(t_u, *params)
    loss = loss_fn(t_p, t_c)
    loss.backward()
    params = (params - learning_rate*params.grad).detach().requires_grad_()
    if epoch%500 == 0:
      print('Epoch %d, Loss %f' % (epoch, float(loss)))
  return params


In [22]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

In [25]:
training_loop(5000, 1e-2, params, 0.1*t_u, t_c)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

In [ ]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from  torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import torchvision


batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=batch_size, shuffle=True)

In [ ]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

device = torch.device('cuda:0')  #初始化GPU
net = MLP().to(device)       #初始化类并将其全连接类分配到GPU
optimizer = optim.SGD(net.parameters(), lr=learning_rate) #初始化优化器分配参数
criteon = nn.CrossEntropyLoss().to(device)    # 初始化损失函数并分配给GPU

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()  #初始化数据并分配给gpu

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.305637
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.010831
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.401249

Test set: Average loss: 0.0047, Accuracy: 7754/10000 (78%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.006127
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.887509
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.798570

Test set: Average loss: 0.0032, Accuracy: 8120/10000 (81%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.643918
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.496492
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.367412

Test set: Average loss: 0.0018, Accuracy: 8991/10000 (90%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.383935
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.324373
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.331903

Test set: Average loss: 0.0016, Accuracy: 9064/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.365042
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.333837
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.254854

Test set: Average loss: 0.0015